In [8]:
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import re
import sqlite3

In [2]:
resto = ["https://www.crous-strasbourg.fr/restaurant/cafeteria-le-pege-2/",
         "https://www.crous-strasbourg.fr/restaurant/resto-u-gallia-2/",
         "https://www.crous-strasbourg.fr/restaurant/resto-u-esplanade-2/",
         "https://www.crous-strasbourg.fr/restaurant/resto-u-paul-appell/",
         "https://www.crous-strasbourg.fr/restaurant/le-32-2/",
         "https://www.crous-strasbourg.fr/restaurant/lannexe-2/",
         "https://www.crous-strasbourg.fr/restaurant/resto-u-illkirch-2/",
         "https://www.crous-strasbourg.fr/restaurant/cafeteria-mini-r-2/",
         "https://www.crous-strasbourg.fr/restaurant/resto-u-cronenbourg-2/",
         "https://www.crous-strasbourg.fr/restaurant/le-cristal-shop-ru-esplanade-2/"
        ]

#verificare cosa offrono
cafe = ["https://www.crous-strasbourg.fr/restaurant/cafeteria-de-la-fac-de-droit-2/",
        "https://www.crous-strasbourg.fr/restaurant/cafeteria-le-cardo-2/",
        "https://www.crous-strasbourg.fr/restaurant/le-stift/",
        "https://www.crous-strasbourg.fr/restaurant/cafeteria-la-misha-2/", #indirizzo: All. du Général Rouvillois, 67000 Strasbourg
        "https://www.crous-strasbourg.fr/restaurant/cafeteria-le-patio-2/",
        "https://www.crous-strasbourg.fr/restaurant/cafeteria-mini-r-2/"
]

**Tabella Info Restourant**

In [3]:
restoInfo = {"hours": [],
            "address": [],
            "payment": [],
            "services": []}

menuInfo = {"address": [],
           "day": [],
           "lunch": [],
           "dinner": []}

driver = webdriver.Firefox()
for i in resto:
    driver.get(i)
    
    element = WebDriverWait(driver, 10).until(
        EC.text_to_be_present_in_element((By.CSS_SELECTOR, 'div.info'), "HORAIRES")
    )

    try:
        reject_all = driver.find_element(By.XPATH, "/html/body/div[6]/div/div[2]/div/div[2]/button[1]")
        reject_all.click()
    except Exception:
        try:
            reject_all = driver.find_element(By.ID, "tru_deselect_btn")
            reject_all.click()
        except Exception:
            pass


    info_div = driver.find_elements(By.CSS_SELECTOR, 'div.info')
    for div in info_div:
        restoInfo['link'] = i
        
        if "HORAIRES" in div.text:
            lines = div.text.split('\n')
            restoInfo["hours"].append(lines[1:])
            
        if "ADRESSE" in div.text:
            lines = div.text.split('\n')
            address = lines[1:][0]
            if "67000" not in address and "67400" not in address and "67200" not in address:
                address = address + " 67000 Strasbourg"
            restoInfo["address"].append(address)        
        
        if "PAIEMENT POSSIBLE" in div.text:
            lines = div.text.split('\n')
            restoInfo["payment"].append(lines[1:])
            
        if "PRATIQUE" in div.text:
            lines = div.text.split('\n')
            restoInfo["services"].append(lines[1:])
           
        
    date = driver.find_elements(By.CSS_SELECTOR, 'time.menu_date_title')
    for j in range(len(date)):
        driver.execute_script("window.scrollBy(0, 300);")
        time.sleep(0.5)
        date = driver.find_elements(By.CSS_SELECTOR, 'time.menu_date_title')

        meal_div = driver.find_elements(By.CSS_SELECTOR, 'div.meal')
        x = []
        for meal in meal_div:
            if meal.text != "":
                x.append(meal.text)

        menuInfo["address"].append(address)
        menuInfo["day"].append(' '.join(date[j].text.split()[2:5]))
        
        if len(x) == 2:
            menuInfo["lunch"].append(x[0])
            menuInfo["dinner"].append(x[1])
        elif len(x) == 1:
            menuInfo["lunch"].append(x[0])
            menuInfo["dinner"].append("")
        elif len(x) == 0:
            menuInfo["lunch"].append("")
            menuInfo["dinner"].append("")
        

        next_button = driver.find_element(By.XPATH, f"/html/body/article/div/section[1]/div/div/div[{j+1}]/div[1]/div[2]")
        next_button.click()

driver.quit()

restoInfo = pd.DataFrame(restoInfo)
menuInfo = pd.DataFrame(menuInfo)

NoSuchWindowException: Message: Browsing context has been discarded
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:193:5
NoSuchWindowError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:679:5
assert.that/<@chrome://remote/content/shared/webdriver/Assert.sys.mjs:515:13
assert.open@chrome://remote/content/shared/webdriver/Assert.sys.mjs:147:4
GeckoDriver.prototype.findElement@chrome://remote/content/marionette/driver.sys.mjs:1539:15
despatch@chrome://remote/content/marionette/server.sys.mjs:318:40
execute@chrome://remote/content/marionette/server.sys.mjs:289:16
onPacket/<@chrome://remote/content/marionette/server.sys.mjs:262:20
onPacket@chrome://remote/content/marionette/server.sys.mjs:263:9
_onJSONObjectReady/<@chrome://remote/content/marionette/transport.sys.mjs:494:20


In [ ]:
restoInfo

In [ ]:
menuInfo

In [ ]:
# Funzione per gestire il click su eventuali popup
def close_popups(driver):
    try:
        reject_all = driver.find_element(By.XPATH, "/html/body/div[6]/div/div[2]/div/div[2]/button[1]")
        reject_all.click()
    except Exception:
        try:
            reject_all = driver.find_element(By.ID, "tru_deselect_btn")
            reject_all.click()
        except Exception:
            pass

# Funzione per estrarre informazioni generali del ristorante
def extract_restaurant_info(info_div):
    resto_data = {"hours": [], "address": [], "payment": [], "services": []}
    
    for div in info_div:
        text = div.text
        if "HORAIRES" in text:
            lines = text.split('\n')
            resto_data["hours"].append(lines[1:])
        
        if "ADRESSE" in text:
            lines = text.split('\n')
            address = lines[1]
            if not any(code in address for code in ["67000", "67400", "67200"]):
                address += " 67000 Strasbourg"
            resto_data["address"].append(address)

        if "PAIEMENT POSSIBLE" in text:
            lines = text.split('\n')
            resto_data["payment"].append(lines[1:])
        
        if "PRATIQUE" in text:
            lines = text.split('\n')
            resto_data["services"].append(lines[1:])
    
    return resto_data

# Funzione per estrarre i menù
def extract_menu_info(driver, address):
    menu_data = {"address": [], "day": [], "lunch": [], "dinner": []}
    
    # Estrazione delle date dei menù
    date_elements = driver.find_elements(By.CSS_SELECTOR, 'time.menu_date_title')
    
    for j, date_elem in enumerate(date_elements):
        # Scroll migliorato tramite script
        driver.execute_script("window.scrollBy(0, 300);")
        WebDriverWait(driver, 5).until(
            EC.visibility_of(date_elem)
        )

        # Estrarre le informazioni dei pasti
        meal_div = driver.find_elements(By.CSS_SELECTOR, 'div.meal')
        meals = [meal.text for meal in meal_div if meal.text]

        menu_data["address"].append(address)
        menu_data["day"].append(' '.join(date_elem.text.split()[2:5]))
        
        if len(meals) == 2:
            menu_data["lunch"].append(meals[0])
            menu_data["dinner"].append(meals[1])
        elif len(meals) == 1:
            menu_data["lunch"].append(meals[0])
            menu_data["dinner"].append("")
        else:
            menu_data["lunch"].append("")
            menu_data["dinner"].append("")
        
        # Gestione del click sul bottone per il menù successivo
        try:
            next_button = driver.find_element(By.XPATH, f"/html/body/article/div/section[1]/div/div/div[{j+1}]/div[1]/div[2]")
            next_button.click()
        except Exception as e:
            pass  # Se non c'è il bottone, passiamo semplicemente al prossimo
    
    return menu_data

# Impostazioni iniziali
restoInfo = {"hours": [], "address": [], "payment": [], "services": []}
menuInfo = {"address": [], "day": [], "lunch": [], "dinner": []}

driver = webdriver.Firefox()

# Iterazione sui ristoranti
for i in resto:
    driver.get(i)
    
    # Attesa della presenza del div con le informazioni del ristorante
    WebDriverWait(driver, 10).until(
        EC.text_to_be_present_in_element((By.CSS_SELECTOR, 'div.info'), "HORAIRES")
    )
    
    # Chiusura popup
    close_popups(driver)
    
    # Estrazione delle informazioni del ristorante
    info_div = driver.find_elements(By.CSS_SELECTOR, 'div.info')
    resto_data = extract_restaurant_info(info_div)
    
    # Aggiornamento dei dati del ristorante
    restoInfo["hours"].extend(resto_data["hours"])
    restoInfo["address"].extend(resto_data["address"])
    restoInfo["payment"].extend(resto_data["payment"])
    restoInfo["services"].extend(resto_data["services"])
    
    # Estrazione dell'indirizzo corrente (sarà usato per i menù)
    current_address = resto_data["address"][-1] if resto_data["address"] else ""

    # Estrazione delle informazioni sui menù
    menu_data = extract_menu_info(driver, current_address)
    
    # Aggiornamento dei dati dei menù
    menuInfo["address"].extend(menu_data["address"])
    menuInfo["day"].extend(menu_data["day"])
    menuInfo["lunch"].extend(menu_data["lunch"])
    menuInfo["dinner"].extend(menu_data["dinner"])

# Chiudere il driver
driver.quit()

# Creazione dei DataFrame
restoInfo = pd.DataFrame(restoInfo)
menuInfo = pd.DataFrame(menuInfo)


# Function resturant information

In [4]:
restoInfo = {"name" : [], "link" : [], "hours": [], "address": [], "payment": [], "services": []}

def ScrapeInfo():
    driver = webdriver.Firefox()
    
    for i in resto:
        driver.get(i)

        element = WebDriverWait(driver, 10).until(
            EC.text_to_be_present_in_element((By.CSS_SELECTOR, 'div.info'), "HORAIRES")
        )

        try:
            reject_all = driver.find_element(By.XPATH, "/html/body/div[6]/div/div[2]/div/div[2]/button[1]")
            reject_all.click()
        except Exception:
            try:
                reject_all = driver.find_element(By.ID, "tru_deselect_btn")
                reject_all.click()
            except Exception:
                pass

            
        restoInfo["name"].append(re.search(r'/([a-zA-Z0-9\-]+?)(?:-\d+)?/$', i).group(1).replace("-"," "))
            
        restoInfo['link'].append(i)

        info_div = driver.find_elements(By.CSS_SELECTOR, 'div.info')
        for div in info_div:
            
            if "HORAIRES" in div.text:
                lines = div.text.split('\n')
                restoInfo["hours"].append(" ".join(lines[1:]))

            if "ADRESSE" in div.text:
                lines = div.text.split('\n')
                address = lines[1:][0]
                if "67000" not in address and "67400" not in address and "67200" not in address:
                    address = address + " 67000 Strasbourg"
                restoInfo["address"].append(address)        

            if "PAIEMENT POSSIBLE" in div.text:
                lines = div.text.split('\n')
                restoInfo["payment"].append(", ".join(lines[1:]))

            if "PRATIQUE" in div.text:
                lines = div.text.split('\n')
                restoInfo["services"].append(", ".join(lines[1:]))

    driver.quit()

    return restoInfo
    
    

In [5]:
ScrapeInfo()

{'name': ['cafeteria le pege',
  'resto u gallia',
  'resto u esplanade',
  'resto u paul appell',
  'le 32',
  'lannexe',
  'resto u illkirch',
  'cafeteria mini r',
  'resto u cronenbourg',
  'le cristal shop ru esplanade'],
 'link': ['https://www.crous-strasbourg.fr/restaurant/cafeteria-le-pege-2/',
  'https://www.crous-strasbourg.fr/restaurant/resto-u-gallia-2/',
  'https://www.crous-strasbourg.fr/restaurant/resto-u-esplanade-2/',
  'https://www.crous-strasbourg.fr/restaurant/resto-u-paul-appell/',
  'https://www.crous-strasbourg.fr/restaurant/le-32-2/',
  'https://www.crous-strasbourg.fr/restaurant/lannexe-2/',
  'https://www.crous-strasbourg.fr/restaurant/resto-u-illkirch-2/',
  'https://www.crous-strasbourg.fr/restaurant/cafeteria-mini-r-2/',
  'https://www.crous-strasbourg.fr/restaurant/resto-u-cronenbourg-2/',
  'https://www.crous-strasbourg.fr/restaurant/le-cristal-shop-ru-esplanade-2/'],
 'hours': ['07h30 à 14h30',
  'Ouvert du lundi au vendredi  > Self et Brasserie : 11h30 

In [6]:
restoInfo = pd.DataFrame(restoInfo)

In [9]:
conn = sqlite3.connect('info.db')

# Scrivi il DataFrame in una tabella SQLite
restoInfo.to_sql('my_table', conn, if_exists='replace', index=False)

# Chiudi la connessione
conn.close()